In [1]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

import s3fs
import os
import torch
import random
import json
# import sktime

from torch import nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from redshift import get_from_redshift
from datetime import datetime
from config import Config  
from collections import defaultdict, Counter
from tqdm import tqdm 
from itertools import zip_longest

tqdm.pandas()

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Loading data

In [2]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_work_experience) 

In [3]:
candidate_work_experience = get_from_redshift(sql_path, Config.redshift_creds)
candidate_work_experience.head()

,candidate_id,unique_id,date_start_job,date_end_job,function_id,isco_functie_niveau,isco_code4,function_name_self,company_name,source
0,96481,214043,1999-07-27 00:00:00,1999-08-23,2038.0,1.0,9329.0,Functienaam van code 9970,Drai Display Europe BV,plaatsing
1,96481,2192150,2000-08-14 00:00:00,2000-08-14,2038.0,1.0,9329.0,Functienaam van code 9970,Banta Global Turnkey B.V.,plaatsing
2,96481,215214,2000-08-16 00:00:00,2000-09-18,2038.0,1.0,9329.0,Functienaam van code 9970,Kamp Coating Apeldoorn BV,plaatsing
3,96481,246438,2004-08-16 00:00:00,2004-08-23,2038.0,1.0,9329.0,Productie-medew uitz,Coldec Productie B.V.,plaatsing
4,96481,2182290,2004-09-13 00:00:00,2005-03-31,2439.0,1.0,9321.0,None,Banta Global Turnkey B.V.,plaatsing


In [4]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_education) 

In [5]:
candidate_education = get_from_redshift(sql_path, Config.redshift_creds)
candidate_education.head()

,candidate_id,education_level,date_start,date_end,date_registrated,passed
0,239187,3,1985-09-01 00:00:00,1987-06-01 00:00:00,2005-08-16 16:53:06,1
1,109774,2,2001-09-01 00:00:00,2004-09-01 00:00:00,2005-07-26 17:50:44,0
2,179574,3,2000-09-01 00:00:00,2001-06-01 00:00:00,2005-08-09 18:28:17,1
3,239357,2,1998-09-01 00:00:00,2002-06-08 00:00:00,2005-08-16 16:53:40,1
4,294921,3,2001-09-01 00:00:00,2005-08-31 00:00:00,2005-08-17 13:12:58,1


In [6]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_skills) 

#### What's date start here? 

In [7]:
candidate_skill = get_from_redshift(sql_path, Config.redshift_creds)
candidate_skill.head()

,candidate_id,skill_id,date_start
0,6854077,2939,2016-12-12 00:51:43
1,2364272,73,2010-03-09 18:40:54
2,6572122,3019,2017-02-22 15:33:17
3,5242879,2975,2014-04-16 09:23:23
4,5980234,3017,2016-02-08 13:40:19


# Skill reindexing

In [8]:
skill_to_idx = {v : i for i, v in enumerate(sorted(candidate_skill["skill_id"].astype(float).unique()))}
idx_to_skill = {i : v for i, v in skill_to_idx.items()}
candidate_skill["skill_id"] = candidate_skill["skill_id"].replace(skill_to_idx)
skills_dict = candidate_skill.groupby("candidate_id")["skill_id"].apply(list).to_dict()

In [9]:
# TODO: do this in a way that doesn't require 70gb of memory lol
# Convert lists of skills to dataframe of skills per candidate
skills_ct = pd.crosstab(candidate_skill["candidate_id"], 
                        candidate_skill["skill_id"])

# # skills = skills_ct.where(skills_ct != 1, skills_ct.columns.to_series(), axis=1)
skills_ct.columns = [f"skill_{i}" for i in skills_ct.columns]

In [10]:
skills_ct.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 465522 entries, 84267 to 7980693
Columns: 317 entries, skill_0 to skill_316
dtypes: int64(317)
memory usage: 1.1 GB


In [11]:
# skills_ct.to_csv("skills_one-hot.csv")

# Merging and formatting

In [12]:
def clean_df(df, table = "work_experience"):
    
    if table == "work_experience":
        start, end = "date_start_job", "date_end_job"
    elif table == "education":
        start, end = "date_start", "date_end"
    else:
        return NotImplemented
    
    # Drop nonsense data
    df = df[df[start] >= dt.datetime(1800, 1, 1)]
    df = df[df[end] <= dt.datetime(2100, 1, 1)]

    # Convert datetime to date
    df[start] = pd.to_datetime(df[start]).dt.date
    df[end] = pd.to_datetime(df[end]).dt.date
    
    return df
        
candidate_work_experience = clean_df(candidate_work_experience)
candidate_education = clean_df(candidate_education, table = "education")

In [13]:
candidate_work_experience["time_spent"] = (candidate_work_experience["date_end_job"] - 
                                           candidate_work_experience["date_start_job"]).dt.days.astype('int16')

In [14]:
def find_educations(df_work, df_education):

    # Merge career data with education levels
    career_education = pd.merge(df_work, 
                                df_education, 
                                on = "candidate_id", 
                                how = "left")[["candidate_id",
                                               "date_start_job",
                                               "date_end_job",
                                               "time_spent",
                                               "education_level",
                                               "date_start",
                                               "date_end",
                                               "passed"]]
    
    # Filter out education that were not passed (yet) at the time of starting a job
    passed = career_education[(career_education["date_start_job"] >= career_education["date_end"]) & 
                              (career_education["passed"] == 1)]
    
    # Only store the highest education level reached at the start of each job
    education_through_time = passed.groupby(["candidate_id", "date_start_job"])["education_level"].max()
    
    df_work.set_index(["candidate_id", "date_start_job"], inplace=True)

    # Store education data in candidate_work_experience
    df_work["education"] = education_through_time

    # np.nan education = no education
    df_work["education"].fillna(0, inplace=True)

    # Reset index for further data augmentation
    df_work.reset_index(inplace=True)
        
    return df_work

candidate_work_experience = find_educations(candidate_work_experience, candidate_education)

candidate_work_experience.head()

,candidate_id,date_start_job,unique_id,date_end_job,function_id,isco_functie_niveau,isco_code4,function_name_self,company_name,source,time_spent,education
0,96481,1999-07-27,214043,1999-08-23,2038.0,1.0,9329.0,Functienaam van code 9970,Drai Display Europe BV,plaatsing,27,0.0
1,96481,2000-08-14,2192150,2000-08-14,2038.0,1.0,9329.0,Functienaam van code 9970,Banta Global Turnkey B.V.,plaatsing,0,0.0
2,96481,2000-08-16,215214,2000-09-18,2038.0,1.0,9329.0,Functienaam van code 9970,Kamp Coating Apeldoorn BV,plaatsing,33,0.0
3,96481,2004-08-16,246438,2004-08-23,2038.0,1.0,9329.0,Productie-medew uitz,Coldec Productie B.V.,plaatsing,7,3.0
4,96481,2004-09-13,2182290,2005-03-31,2439.0,1.0,9321.0,None,Banta Global Turnkey B.V.,plaatsing,199,3.0


In [15]:
# Add skills
# candidate_work_experience = pd.merge(candidate_work_experience, 
#                                      skills_ct, 
#                                      left_on="candidate_id", 
#                                      right_index=True, 
#                                      how="left")

# Filtering and reindexing

In [16]:
more_than_5 = candidate_work_experience["isco_code4"].value_counts()
more_than_5 = set(more_than_5[more_than_5 > 5].index)
candidate_work_experience = candidate_work_experience[candidate_work_experience["isco_code4"].isin(more_than_5)]

In [ ]:
id_to_idx = {v : i for i, v in enumerate(sorted(candidate_work_experience["function_id"].astype(float).unique()))}
idx_to_id = {i : v for i, v in id_to_idx.items()}

code_to_idx = {v : i for i, v in enumerate(sorted(candidate_work_experience["isco_code4"].astype(float).unique()))}
idx_to_code = {i : v for i, v in code_to_idx.items()}

In [ ]:
candidate_work_experience["function_id"] = candidate_work_experience["function_id"].replace(id_to_idx)
candidate_work_experience["isco_code4"] = candidate_work_experience["isco_code4"].replace(code_to_idx)

In [17]:
candidate_work_experience.shape

(1666709, 12)

In [18]:
# Calculate the time it took to go from one job to another (in order)
candidate_work_experience["time_between"] = candidate_work_experience.groupby(
    "candidate_id")["date_start_job"].progress_apply(lambda x: x - x.shift(1))

100%|██████████| 468039/468039 [04:25<00:00, 1766.18it/s]


# Add CV embeddings

In [21]:
embedding_pd = pd.read_parquet("s3://s3-nl-prd-semrb-emr/embeddings/doc_embeddings/word2vec/word2vec_doc_embedding.parquet")
embedding_pd['cv_id'] = embedding_pd['cv_id'].astype('int')
embedding_pd.rename(columns={"doc_embedding": "tensor"}, inplace=True)

In [22]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_cvs) 

candidate_cvs = get_from_redshift(sql_path, Config.redshift_creds)

In [23]:
def find_embeddings(embedding_pd, candidate_cvs):
    embedding_pd = embedding_pd.set_index("cv_id")
    
    candidate_cvs = candidate_cvs[["cv_id", "candidate_id", "date_start"]]

    embedding_per_candidate = pd.merge(embedding_pd, 
                                       candidate_cvs, 
                                       left_index=True, 
                                       right_on="cv_id")[["candidate_id", "embedding", "date_start"]]
    
    matches = (set(embedding_per_candidate["candidate_id"]) & set(candidate_work_experience["candidate_id"]))
    
    matched_cvs = embedding_per_candidate[embedding_per_candidate["candidate_id"].isin(matches)]
    matched_cvs["date_start"] = matched_cvs["date_start"].dt.date
    
    last_cv_per_day = matched_cvs.groupby(["candidate_id", "date_start"])["embedding"].apply(lambda x: list(x)[-1])
    
    cv_embeddings = last_cv_per_day.reset_index()
    
    return cv_embeddings

cv_embeddings = find_embeddings(embedding_pd, candidate_cvs)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
cv_embeddings.head()

,candidate_id,date_start,embedding
0,84556.0,2020-07-02,"[0.07265971601009369, -0.8582056760787964, -0...."
1,84612.0,2020-05-30,"[0.3474675714969635, -1.0429900884628296, -0.3..."
2,84612.0,2020-06-03,"[0.34068500995635986, -1.0179638862609863, -0...."
3,84731.0,2020-06-03,"[0.12455234676599503, -0.1725415289402008, -0...."
4,85437.0,2020-06-06,"[0.2048000991344452, -0.49068811535835266, -0...."


In [25]:
def add_embeddings(candidate_work_experience, cv_embeddings):
    
    # Add CVs to candidates (includes duplicates)
    full_merge = pd.merge(candidate_work_experience,
                          cv_embeddings,
                          left_on="candidate_id",
                          right_on="candidate_id",
                          how="left")
    
    display(full_merge)
    
    # Find the maximum date of uploaded cvs per job (i.e. most recent CV during each job)
    most_recent_cvs = full_merge[(full_merge["date_start_job"]
                                  >= full_merge["date_start"])].groupby(
        "unique_id")["date_start"].idxmax()   
    
    # Some candidates only added CVs after their last job started, so account for that too
    late_cvs = full_merge[(full_merge["date_start_job"] <= 
                           full_merge["date_start"])].groupby("unique_id")["date_start"].idxmax()
    
    # Filter out everything we already found earlier
    late_cvs = late_cvs[set(late_cvs.index) - set(most_recent_cvs.index)]
    
    # Now we have all the CVs in one place
    cv_idxs = pd.concat([most_recent_cvs, late_cvs])
    
    # Combine the two frames to include candidates without any CVs
    combined = pd.concat([full_merge[full_merge["embedding"].isna()], 
                          full_merge.loc[cv_idxs.values]])
    
    return combined

candidate_work_experience = add_embeddings(candidate_work_experience, cv_embeddings)

,candidate_id,date_start_job,unique_id,date_end_job,function_id,isco_functie_niveau,isco_code4,function_name_self,company_name,source,time_spent,education,time_between,date_start,embedding
0,96481,1999-07-27,214043,1999-08-23,2038.0,1.0,9329.0,Functienaam van code 9970,Drai Display Europe BV,plaatsing,27,0.0,NaT,NaT,NaN
1,96481,2000-08-14,2192150,2000-08-14,2038.0,1.0,9329.0,Functienaam van code 9970,Banta Global Turnkey B.V.,plaatsing,0,0.0,384 days,NaT,NaN
2,96481,2000-08-16,215214,2000-09-18,2038.0,1.0,9329.0,Functienaam van code 9970,Kamp Coating Apeldoorn BV,plaatsing,33,0.0,2 days,NaT,NaN
3,96481,2004-08-16,246438,2004-08-23,2038.0,1.0,9329.0,Productie-medew uitz,Coldec Productie B.V.,plaatsing,7,3.0,1461 days,NaT,NaN
4,96481,2004-09-13,2182290,2005-03-31,2439.0,1.0,9321.0,None,Banta Global Turnkey B.V.,plaatsing,199,3.0,28 days,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863012,8897546,2022-04-15,10582043,2023-04-16,292.0,1.0,9621.0,Courier,Takeaway.com Express Netherlands B.V.,plaatsing,366,0.0,NaT,NaT,NaN
1863013,8897547,2017-02-01,17680797,2018-04-01,292.0,1.0,9621.0,Bezorger,Dominos,dossier,424,0.0,NaT,NaT,NaN
1863014,8897799,2021-08-01,17680902,2021-12-01,2038.0,1.0,9329.0,Productiemedewerker,"""V i r g i n N u t s""",dossier,122,0.0,NaT,NaT,NaN
1863015,8897803,2016-02-01,17680904,2016-11-01,1092.0,2.0,5120.0,Zelfstandig werkende kok,Van der Valk,dossier,274,0.0,NaT,NaT,NaN


In [26]:
# candidate_work_experience["embedding"] = candidate_work_experience["embedding"].apply(lambda x: x if x is not np.nan
#                                                                                                   else [0] * 300)

In [27]:
candidate_work_experience = candidate_work_experience.reset_index().drop("index", axis=1)

In [28]:
# embedding_lists = candidate_work_experience["embedding"].to_list()
# embeddings = pd.DataFrame(embedding_lists)

In [29]:
# embeddings.columns = [f"embedding_{i}" for i in range(len(embeddings.columns))]

In [30]:
# candidate_work_experience = pd.concat([candidate_work_experience, embeddings], axis=1)

# Ordering & Time

In [31]:
def add_order(df):
    
    # Count the number of jobs each candidate has ahd
    job_counts = df.groupby("candidate_id").size()  
    
    # Sort by candidate_id, date_start_job
    sorted_df = df.sort_values(by = ['candidate_id', "date_start_job"])
    
    # Reset index
    sorted_df.reset_index(inplace=True, drop=True)
    
    # Create a list of lists containing the order of each candidates jobs (which came first, second, third, etc.)
    order = [np.arange(count) for count in job_counts.values]
    
    # Flatten list
    order = [item for sublist in order for item in sublist]
    
    # Add order to df
    sorted_df["job_order"] = order
    
    # Set a candidate_id, job_order as the index
    return sorted_df.set_index(["candidate_id", 
                                "job_order"])

df = add_order(candidate_work_experience)

In [32]:
df = df[["date_start_job", "date_end_job", "time_spent", "time_between", "isco_code4", "function_id",
         "isco_functie_niveau", "company_name", "source", "education", "embedding"]]

df["time_between"] = df["time_between"].shift(-1).fillna(pd.Timedelta(seconds=0)).dt.days.astype('int16')
df.head()

date_start_job date_end_job  time_spent  time_between  \
candidate_id job_order                                                         
84556        0             2000-08-01   2001-01-04         156            37   
             1             2000-09-07   2001-01-01         116            23   
             2             2000-09-30   2001-07-02         275            23   
             3             2000-10-23   2000-10-27           4          1155   
             4             2003-12-22   2004-01-02          11           203   

                        isco_code4  function_id  isco_functie_niveau  \
candidate_id job_order                                                 
84556        0              5131.0       1236.0                  2.0   
             1              9412.0       1065.0                  1.0   
             2              5131.0       1236.0                  2.0   
             3              9329.0       2038.0                  1.0   
             4              9329.0       2038.0                  1.0   

                                                    company_name     source  \
candidate_id job_order                                                        
84556        0          (oud 3146Amsterdam RAI Catering Services  plaatsing   
             1          (oud 3146Amsterdam RAI Catering Services  plaatsing   
             2              AMSTERDAM RAI CATERING SERVICES B.V.  plaatsing   
             3                                 Formido Bouwmarkt  plaatsing   
             4                             Yakult Nederland B.V.  plaatsing   

                        education  \
candidate_id job_order              
84556        0                0.0   
             1                0.0   
             2                0.0   
             3                0.0   
             4                0.0   

                                                                embedding  
candidate_id job_order                                                     
84556        0          [0.07265971601009369, -0.8582056760787964, -0....  
             1          [0.07265971601009369, -0.8582056760787964, -0....  
             2          [0.07265971601009369, -0.8582056760787964, -0....  
             3          [0.07265971601009369, -0.8582056760787964, -0....  
             4          [0.07265971601009369, -0.8582056760787964, -0....

In [33]:
num_classes = len(df["isco_code4"].unique())

In [34]:
# df_pred = df[["isco_functie_niveau", "education", "function_id"]].fillna(0)
pred_cols = ["time_between", "time_spent", "isco_functie_niveau", 
             "education", "function_id", "isco_code4"] # + [col for col in df.columns if "skill_" in col]
num_features = len(pred_cols)

df_pred = df[pred_cols].fillna(0)

In [35]:
df_pred.head()

time_between  time_spent  isco_functie_niveau  \
candidate_id job_order                                                  
84556        0                    37         156                  2.0   
             1                    23         116                  1.0   
             2                    23         275                  2.0   
             3                  1155           4                  1.0   
             4                   203          11                  1.0   

                        education  function_id  isco_code4  
candidate_id job_order                                      
84556        0                0.0       1236.0      5131.0  
             1                0.0       1065.0      9412.0  
             2                0.0       1236.0      5131.0  
             3                0.0       2038.0      9329.0  
             4                0.0       2038.0      9329.0

In [38]:
df_pred[["function_id", "isco_code4"]].reset_index().to_csv("DTW_df_pred.csv")

### Create separate embedding dict

Done for optimization purposes

In [ ]:
embedding_order = df.reset_index()[["candidate_id", "job_order", "embedding"]]

In [ ]:
grouped_embeddings = embedding_order.groupby("candidate_id")

In [ ]:
embedding_a = defaultdict(lambda: defaultdict(list))

# Find each time step at which the candidate got a new CV
for candidate, values in tqdm(grouped_embeddings):
    embeddings = values["embedding"].values
  
    if (type(embeddings[0]) == type(np.array([]))) and np.nan not in embeddings[0]:
        embedding_a[candidate][0] = embeddings[0]
        
        for i, embedding in enumerate(embeddings):
            if i > 0:
                truths = embeddings[i - 1] != embedding

                if (type(truths) == type(np.array([]))) and (truths).all():
                    embedding_a[candidate][i] = embedding

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
with open('embeddings.json', 'w') as f:
    json.dump(embedding_a, f, cls=NumpyEncoder)

In [ ]:
df_pred.reset_index().to_csv("df_pred.csv")

In [ ]:
pd.Series(skills_dict).to_csv("skills.csv")

### Candidate Certificates

In [ ]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_certificates) 

candidate_certificates = get_from_redshift(sql_path, Config.redshift_creds)
candidate_certificates.head()

In [ ]:
candidate_certificates = pd.crosstab(candidate_certificates["candidate_id"], 
                                     candidate_certificates["candidate_certificate_id"])
candidate_certificates.head()

In [ ]:
candidate_certificates.to_csv("candidate_certificates_one-hot.csv")

### CV Embeddings

In [ ]:
df_pred = pd.read_csv("df_pred.csv")
skills = pd.read_csv("skills.csv")

In [ ]:
# s3 = s3fs.S3FileSystem()

# embedding_pd = pq.ParquetDataset('s3a://s3-nl-prd-semrb-emr/embeddings/doc_embeddings/xlm-roberta-base-smartmatch', filesystem=s3).read_pandas().to_pandas()

# embedding_pd['cv_id'] = embedding_pd['cv_id'].astype('int')
# embedding_pd.rename(columns={"doc_embedding": "tensor"}, inplace=True)
# embedding_pd.head() 

In [ ]:
# cvPath = 's3a://s3-nl-prd-datahub-projects/smartmatch_cv/parsed_cv'

# s3 = s3fs.S3FileSystem()
# parsedCv_pd = pq.ParquetDataset(cvPath, filesystem=s3).read_pandas().to_pandas() 

In [ ]:
# parsedCv_pd

In [ ]:
len(set(cv_embeddings["candidate_id"]) & set(df_pred["candidate_id"]))

In [ ]:
cv_embeddings.to_csv("cv_w2v_embeddings.csv")